## Importing the necessary libraries

In [1]:
import datetime
import pytz
import speech_recognition as sr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json
import numpy as np

In [2]:
## loading the model

model_hi = load_model('english_to_hindi_lstm_model')

#load Tokenizer
with open('english_tokenizer_hindi.json') as f:
    data = json.load(f)
    english_tokenizer = tokenizer_from_json(data)

with open('hindi_tokenizer.json') as f:
    data = json.load(f)
    hindi_tokenizer = tokenizer_from_json(data)

In [4]:
with open('sequence_length_hindi.json') as f:
    max_length_hindi = json.load(f)
    
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

def translate_to_hindi(english_sentence):
    english_sentence = english_sentence.lower()
    
    english_sentence = english_sentence.replace(".", '')
    english_sentence = english_sentence.replace("?", '')
    english_sentence = english_sentence.replace("!", '')
    english_sentence = english_sentence.replace(",", '')
    
    english_sentence = english_tokenizer.texts_to_sequences([english_sentence])
    english_sentence = pad(english_sentence, max_length_hindi)
    
    english_sentence = english_sentence.reshape((-1,max_length_hindi))
    
    hindi_sentence = model_hi.predict(english_sentence)[0]
    
    hindi_sentence = [np.argmax(word) for word in hindi_sentence]

    hindi_sentence = hindi_tokenizer.sequences_to_texts([hindi_sentence])[0]
    
    # print("hindi translation: ", hindi_sentence)
    
    return hindi_sentence

In [13]:
def is_after_6pm_ist():
    ist = pytz.timezone('Asia/Kolkata')
    current_time = datetime.datetime.now(ist)
    return current_time.hour >= 18

def listen_and_translate():
    recognizer = sr.Recognizer()

    while True:
        with sr.Microphone() as source:
            print("Listening...")
            audio = recognizer.listen(source)

        try:
            english_text = recognizer.recognize_google(audio)
            print(f"You said: {english_text}")

            if not is_after_6pm_ist():
                print("Please try after 6 PM IST.")
                return

            if english_text.strip().lower()[0] in ['m', 'o']:
                print("Cannot translate words starting with 'M' or 'O'.")
                return

            hindi_translation = translate_to_hindi(english_text)
            print(f"Hindi translation: {hindi_translation}")
            return

        except sr.UnknownValueError:
            print("Sorry, I couldn't understand the audio. Please try again.")
        except sr.RequestError as e:
            print(f"Could not request results; {e}")
            return

In [12]:
listen_and_translate()

Listening...
You said: origami
Cannot translate words starting with 'M' or 'O'.


#